<a href="https://colab.research.google.com/github/juliodouglas1968/juliodouglas/blob/master/7MAIR_Aprendizaje_por_Refuerzo_Ejercicio_de_evaluaci%C3%B3n1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym

!pip install h5py

!pip install Pillow

!pip install gym[atari]

!pip install keras-rl

!pip install tensorflow==1.13.1
!pip install keras-rl==0.4.2


     |████████████████████████████████| 40kB 2.2MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=1293e7a530a40754497d8f927e310b6df1400afeb68f0f10c17926634feaec61
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 92.5MB 35kB/s 
     |████████████████████████████████| 368kB 45.7MB/s 
     |████████████████████████████████| 3.2MB 36.9MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [2]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
ENV_NAME = 'PongDeterministic-v0'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [0]:
from rl.core import Processor
# Define the input shape to resize the screen
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

# This processor will be similar to the Atari processor
class PongProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        
        img = Image.fromarray(observation)
        # resize and convert to grayscale
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [5]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1612816   
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
________________________________________________

In [0]:
# Let's define the memory for storing the experience
memory = SequentialMemory(limit=50000, window_length=1)

# Define the policy that our agent will follow
policy = BoltzmannQPolicy()


# Define the agent
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
optimizer = Adam(lr=1e-3)
dqn.compile(optimizer, metrics=['mae'])

In [7]:
# Train the agent
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

# After training is done, we save the final weights.
#dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

Training for 50000 steps ...
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   764/50000: episode: 1, duration: 47.968s, episode steps: 764, steps per second: 16, episode reward: -21.000, mean reward: -0.027 [-1.000, 0.000], mean action: 2.616 [0.000, 5.000], mean observation: 98.094 [0.000, 236.000], loss: 2741.843194, mean_absolute_error: 1.774980, mean_q: 3.304803
  1651/50000: episode: 2, duration: 53.377s, episode steps: 887, steps per second: 17, episode reward: -20.000, mean reward: -0.023 [-1.000, 1.000], mean action: 2.454 [0.000, 5.000], mean observation: 98.037 [0.000, 236.000], loss: 2184.275146, mean_absolute_error: 1.888542, mean_q: 5.150588
  2661/50000: episode: 3, duration: 61.778s, episode steps: 1010, steps per second: 16, episode reward: -19.000, mean reward: -0.019 [-1.000, 1.000], mean action: 2.528 [0.000, 5.000], mean observation: 97.989 [0.000, 236.000], loss: 0.012082, mean_absolute_error: 0.060135, mean_q: -0.051341
  3485/50000: episode: 4, duration: 51.156s, episode steps: 824, steps per second: 16, episode reward: -21.000, mean re

In [0]:
# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)